In [2]:
pip install requests beautifulsoup4 pandas openpyxl pykrx lxml

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 크롤러

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pykrx import stock
from urllib.parse import quote
from datetime import datetime

# pandas 출력 설정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

# 사용자 입력 받기
input_name = input("크롤링할 종목명을 입력하세요 (예: 카카오): ").strip()
start_date_input = input("시작일을 입력하세요 (예: 2023-07-01): ").strip()
end_date_input = input("종료일을 입력하세요 (예: 2025-07-25): ").strip()

# 날짜 형식 변환
start_date = datetime.strptime(start_date_input, "%Y-%m-%d")
end_date = datetime.strptime(end_date_input, "%Y-%m-%d")

# 종목명 → 종목코드 변환
ticker_list = stock.get_market_ticker_list(market="ALL")
ticker_name_map = {stock.get_market_ticker_name(code): code for code in ticker_list}

if input_name not in ticker_name_map:
    print(f"[오류] '{input_name}'에 해당하는 종목코드를 찾을 수 없습니다.")
else:
    item_code = ticker_name_map[input_name]
    base_url = "https://finance.naver.com/research/"
    headers = {"User-Agent": "Mozilla/5.0"}

    data = []

    for i in range(1, 7):  # 페이지 수 조정 가능
        url = f"https://finance.naver.com/research/company_list.naver?searchType=itemCode&itemName={quote(input_name)}&itemCode={item_code}&page={i}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        rows = soup.select("table.type_1 tr")[2:]

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 5:
                continue

            a_tag = cols[1].find("a", href=True)
            if not a_tag or not a_tag["href"].startswith("company_read.naver"):
                continue

            title = a_tag.text.strip()
            detail_url = base_url + a_tag["href"]
            broker = cols[2].text.strip()
            pdf_tag = cols[3].find("a", href=True)
            pdf_link = pdf_tag["href"] if pdf_tag and pdf_tag["href"].endswith(".pdf") else None
            date_str = cols[4].text.strip()

            # 작성일 필터링
            try:
                date = datetime.strptime(date_str, "%y.%m.%d")
            except:
                continue

            if not (start_date <= date <= end_date):
                continue

            # 상세 페이지 크롤링 (목표가, 투자의견)
            try:
                detail_res = requests.get(detail_url, headers=headers)
                detail_soup = BeautifulSoup(detail_res.text, "html.parser")

                target_price = detail_soup.select_one("em.money")
                investment_opinion = detail_soup.select_one("em.coment")

                target_price_text = target_price.text.strip() if target_price else None
                opinion_text = investment_opinion.text.strip() if investment_opinion else None
            except:
                target_price_text = None
                opinion_text = None

            data.append({
                "제목": title,
                "상세페이지링크": detail_url,
                "증권사": broker,
                "PDF 링크": pdf_link,
                "작성일": date_str,
                "목표가": target_price_text,
                "투자의견": opinion_text
            })

            time.sleep(0.3)

    # DataFrame 생성 (링크 있는 버전)
    df_link = pd.DataFrame(data)

    # Excel 저장 (링크 있는 버전)
    filename = f"{input_name}_{start_date_input}~{end_date_input}_리포트 수집.xlsx"
    df_link.to_excel(filename, index=False, engine='openpyxl')
    print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

    # DataFrame 생성 (링크 없는 버전)
    df_2 = pd.DataFrame(data)
    df_2 = df_2.drop(columns=["상세페이지링크", "PDF 링크"])


✅ 엑셀 파일로 저장 완료: SK하이닉스_2023-07-01~2024-07-31_리포트 수집.xlsx


# 실제 주가 불러오기

In [2]:
from pykrx import stock
from datetime import timedelta

# 작성일 → datetime 변환
df_2["작성일_dt"] = pd.to_datetime(df_2["작성일"], format="%y.%m.%d")

# 1년 뒤 날짜 계산
df_2["1년뒤_예측일"] = df_2["작성일_dt"] + pd.DateOffset(years=1)

# 종목코드 다시 확인
item_code = ticker_name_map[input_name]

# 1년 뒤 종가 저장용 리스트
actual_prices = []

for idx, row in df_2.iterrows():
    predict_day = row["1년뒤_예측일"]
    from_date = predict_day.strftime("%Y%m%d")
    to_date = (predict_day + timedelta(days=5)).strftime("%Y%m%d")  # 영업일이 아닐 경우 대비
    
    try:
        df_price = stock.get_market_ohlcv_by_date(from_date, to_date, item_code)
        df_price = df_price.reset_index()
        
        # 가장 가까운 날짜의 종가 선택
        if not df_price.empty:
            price = df_price.iloc[0]["종가"]
        else:
            price = None
    except:
        price = None

    actual_prices.append(price)
    time.sleep(0.2)  # 과도한 요청 방지

df_2["1년뒤_실제종가"] = actual_prices


In [3]:
df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가
0,낙폭 과대,SK증권,24.07.29,"340,000",매수,2024-07-29,2025-07-29,262500
1,달라진 건 없다,DS투자증권,24.07.26,"290,000",매수,2024-07-26,2025-07-26,262000
2,"2Q24 Review 컨센상회, 분기 최대 실적",교보증권,24.07.26,"220,000",Buy,2024-07-26,2025-07-26,262000
3,HBM 효과를 지켜보자,하나증권,24.07.26,"280,000",Buy,2024-07-26,2025-07-26,262000
4,2Q24 Review : 너무 높았던 기대. 다만 실적..,한화투자증권,24.07.26,"280,000",Buy,2024-07-26,2025-07-26,262000
...,...,...,...,...,...,...,...,...
79,2Q23 Review: 기술 차별화,유진투자증권,23.07.27,"130,000",Buy,2023-07-27,2024-07-27,195600
80,DRAM 시장 내 입지 강화,유안타증권,23.07.27,"140,000",Buy,2023-07-27,2024-07-27,195600
81,"3분기 영업적자 축소, DRAM 흑자전환 예상",키움증권,23.07.27,"140,000",Buy,2023-07-27,2024-07-27,195600
82,아직 남은 주가 상승,하이투자증권,23.07.27,"132,000",Buy,2023-07-27,2024-07-27,195600


# 오차율, 오차율 절댓값 구하기

In [4]:
# 1. 목표가 문자열 → 수치형으로 변환
def parse_price(price_text):
    if not price_text:
        return None
    try:
        return int(price_text.replace(",", "").replace("원", "").strip())
    except:
        return None

df_2["목표가(수치)"] = df_2["목표가"].apply(parse_price)

# 2. 오차율 계산: (목표가 - 실제종가) / 실제종가 * 100
def calculate_error(target, actual):
    try:
        if pd.notnull(target) and pd.notnull(actual) and actual != 0:
            return round((target - actual) / actual * 100, 2)
        else:
            return None
    except:
        return None

df_2["오차율(%)"] = df_2.apply(lambda row: calculate_error(row["목표가(수치)"], row["1년뒤_실제종가"]), axis=1)

df_2

# 절댓값 기준 오차율 낮은 순서로 정렬
df_2["오차율_절댓값"] = df_2["오차율(%)"].abs()
df_2 = df_2.sort_values("오차율_절댓값", ascending=True)

df_2

,제목,증권사,작성일,목표가,투자의견,작성일_dt,1년뒤_예측일,1년뒤_실제종가,목표가(수치),오차율(%),오차율_절댓값
54,새로운 시작점,미래에셋증권,23.12.29,"173,000",매수,2023-12-29,2024-12-29,173900,173000.0,-0.52,0.52
12,매수기회. 목표가 상향,미래에셋증권,24.07.26,"260,000",매수,2024-07-26,2025-07-26,262000,260000.0,-0.76,0.76
29,HBM 시장 경쟁 심화 예상,키움증권,24.04.26,"180,000",MarketPerform,2024-04-26,2025-04-26,182000,180000.0,-1.10,1.10
6,계획대로 순조롭게 진행 중,대신증권,24.07.26,"265,000",Buy,2024-07-26,2025-07-26,262000,265000.0,1.15,1.15
72,빨라지는 실적 개선의 속도,한화투자증권,23.09.14,"150,000",Buy,2023-09-14,2024-09-14,152800,150000.0,-1.83,1.83
...,...,...,...,...,...,...,...,...,...,...,...
32,1Q24 서프라이즈 예상,SK증권,24.04.15,"250,000",매수,2024-04-15,2025-04-15,180600,250000.0,38.43,38.43
76,2Q23 Review: 장미에는 가시가 있다,이베스트증권,23.07.27,"115,000",Hold,2023-07-27,2024-07-27,195600,115000.0,-41.21,41.21
83,2Q23 Preview: 높아진 기대,이베스트증권,23.07.05,"115,000",Hold,2023-07-05,2024-07-05,236000,115000.0,-51.27,51.27
31,HBM 시장 점유율 하락 예상,키움증권,24.04.19,없음,없음,2024-04-19,2025-04-19,176600,NaN,NaN,NaN


# 증권사별로 모으기

In [5]:
# 1. NaN 제거
df_valid = df_2[df_2["오차율(%)"].notnull()]

# 2. 증권사별 통계 집계
broker_error = (
    df_valid.groupby("증권사")["오차율(%)"]
    .agg([
        ("리포트_수", "count"),
        ("평균_오차율(%)", "mean"),
        ("평균_절댓값_오차율(%)", lambda x: x.abs().mean())
    ])
    .reset_index()
)

# 3. 리포트 수 기준 필터링 (선택 사항)
broker_error = broker_error[broker_error["리포트_수"] >= 1]

# 4. 절댓값 오차율 기준 정렬
broker_error_sorted = broker_error.sort_values("평균_절댓값_오차율(%)")

# 5. 출력
display(broker_error_sorted)


,증권사,리포트_수,평균_오차율(%),평균_절댓값_오차율(%)
6,미래에셋증권,4,1.612500,8.827500
14,한화투자증권,9,-4.595556,12.611111
1,IBK투자증권,3,3.243333,13.016667
4,대신증권,5,4.620000,13.944000
11,키움증권,6,-14.018333,14.018333
3,교보증권,7,-9.347143,15.312857
7,신한투자증권,6,2.958333,15.641667
12,하나증권,7,-4.831429,15.900000
2,SK증권,7,4.871429,16.708571
0,DS투자증권,5,-0.954000,17.430000


# EWM 오차율 구하기

In [6]:
# 1) NaN 제거 + 날짜형
df_valid = df_2[df_2["오차율(%)"].notnull()].copy()
df_valid["작성일_dt"] = pd.to_datetime(df_valid["작성일_dt"])

# 2) 증권사별 EWM(날짜 가중) 계산
HL_DAYS = 90  # 반감기 90일

def calc_ewm(g):
    g = g.sort_values("작성일_dt").copy()
    g["오차율_EWM"] = g["오차율(%)"].abs().ewm(
        halflife=pd.Timedelta(days=HL_DAYS),
        times=g["작성일_dt"],
        adjust=True   # ← ★ 여기! times를 쓰면 adjust=True 이어야 함
    ).mean()
    return g

df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)

# 3) 증권사별 최종 요약(가장 최신 EWM을 대표값으로)
broker_error = (
    df_ewm.sort_values("작성일_dt")
          .groupby("증권사")
          .agg(
              리포트_수=("오차율(%)", "count"),
              EWM_오차율=("오차율_EWM", "last")
          )
          .sort_values("EWM_오차율")
          .reset_index()
)

display(broker_error)



C:\Users\chloe\AppData\Local\Temp\ipykernel_36216\93774328.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ewm = df_valid.groupby("증권사", group_keys=False).apply(calc_ewm)


,증권사,리포트_수,EWM_오차율
0,미래에셋증권,4,7.467679
1,대신증권,5,9.573849
2,한화투자증권,9,11.111652
3,유안타증권,4,11.558879
4,하나증권,7,12.004381
5,IBK투자증권,3,13.209825
6,DS투자증권,5,13.798581
7,현대차증권,3,14.151090
8,유진투자증권,6,14.871645
9,신한투자증권,6,15.083407
